# Extracting job descriptions online

In [1]:
import os
os.chdir('..')

Reading in the necessary data

In [15]:
import pandas as pd
SSOC_2020 = pd.read_csv('Data/Processed/Training/train-aws/SSOC_2020.csv')
data = pd.read_csv('Data/Processed/Training/train-aws/train_full.csv')
extra_info = pd.read_csv('Data/Processed/MCF_Training_Set_Full.csv')
SSOC_2020_Full_data = pd.read_excel('Data/Processed/Training/train-aws/SSOC2020_Detailed_Definitions.xlsx', skiprows=range(0,4))

C:\Users\benjamin\Desktop\my_enviro\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\benjamin\Desktop\my_enviro\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [26]:
SSOC_2020_Full_data = SSOC_2020_Full_data[['SSOC 2020', 'SSOC 2020 Title']]
SSOC_2020_Full_data['SSOC 2020'] = SSOC_2020_Full_data['SSOC 2020'].astype(str)

In [30]:
SSOC_2020['SSOC 2020'] = SSOC_2020['SSOC 2020'].astype(str)
SSOC_2020_m = pd.merge(SSOC_2020, SSOC_2020_Full_data, how = 'left', on = 'SSOC 2020') 

In [46]:
SSOC_2020_unique = SSOC_2020_m[~SSOC_2020_m['SSOC 2020'].isin(data['SSOC 2020'].unique())]
SSOC_2020_unique

,SSOC 2020,Description,SSOC 2020 Title
0,11110,"Legislator determines, formulates and directs ...",Legislator
1,11121,"Senior government official plans, organises an...",Senior government official
2,11122,"Senior statutory board official plans, organis...",Senior statutory board official
3,11140,Senior official of political party organisatio...,Senior official of political party organisation
4,11150,"Senior official of employers', workers' and ot...","Senior official of employers', workers' and ot..."
...,...,...,...
986,96257,Restroom attendant cleans and maintains toilet...,Restroom attendant
990,96269,This group includes watchmen and related worke...,Other watchmen and related workers (e.g. light...
991,96271,Chief concierge (hotel) manages all aspects of...,Chief concierge (hotel)
992,96272,Concierge (hotel) serves as the point of conta...,Concierge (hotel)


In [50]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

Finding top related links in google

In [51]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [91]:
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

In [245]:
scrape_google('Legislator Job Description')

['https://www.jkentstaffing.com/job_title/legislative-assistant/',
 'https://www.jobbank.gc.ca/marketreport/occupation/2913/ca',
 'https://laddc.org/initiatives/community-living-and-self-determination/quality-assurance/current-initiatives/advocacy-101/government-the-legislative-process/the-role-of-a-legislator/',
 'https://bestaccreditedcolleges.org/articles/state-legislator-job-description-duties-and-requirements.html',
 'https://www.owlguru.com/career/legislators/job-description/',
 'https://study.com/articles/Legislative_Analyst_Job_Description_Duties_and_Salary.html',
 'https://www.careerexplorer.com/careers/legislator/',
 'https://www.mightyrecruiter.com/job-descriptions/legislator/',
 'https://www.yourfreecareertest.com/legislator/',
 'https://learn.org/articles/State_Legislator_Career_and_Salary_FAQs.html']

Testing out extracting text using a list of pre-definend titles and threshold

In [176]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup, NavigableString, Tag

In [239]:
def extract_Text(URL, list_of_titles, threshold):
    
    req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, "html.parser")

    tags = ["h3", "h2"]

    for tag in tags:

        python_jobs = soup.find_all(tag)

        actual_text_ref = list_of_titles
        job_ref = [txt.text.lower() for txt in python_jobs]
        
        print(f'List of titles present are: {job_ref}')

        idx, proceede = find_similar(actual_text_ref, job_ref, threshold)

        if proceede:

            text = []

            for header in python_jobs[idx:idx+2]:
                nextNode = header
                while True:
                    nextNode = nextNode.nextSibling
                    if nextNode is None:
                        break
                    if isinstance(nextNode, NavigableString):
                        nextNode.strip()
                    if isinstance(nextNode, Tag):
                        if nextNode.name == tag:
                            break
                        txt = nextNode.get_text(strip=True).strip()
                        if txt:
                            text.append(txt)
            return text
        
        return None

In [229]:
def find_similar(list_of_ref, job_ref, threshold):
    for i in range(len(job_ref)):
        
        job_text = job_ref[i]
        
        for ref in list_of_ref:
            
            if str_similarity(job_text, ref) > threshold:
                return i, True
    
    return None, False

In [225]:
import spacy

nlp = spacy.load('en_core_web_lg')

def str_similarity(string1, string2):
    return nlp(string1).similarity(nlp(string2))

In [240]:
extract_Text(URL, ['job description', 'job duties'], 0.5)

['Legislators participate in the activities of a federal, provincial, territorial or local government legislative body or executive council, band council or school board as elected or appointed members.',
 'This group performs some or all of the following duties:Enact, amend or repeal laws and regulationsParticipate in developing or amending government policies, programs or proceduresRepresent their government at local, national and international meetings and conferencesRespond to matters of concern to constituents or the general publicMay serve as minister responsible for a government department or agency and direct senior government managers and officials in the implementation of government policy and the management of that department or agency.']

Putting all the functions together

In [248]:
def extracting_job_description(Job_title):
    query = Job_title + "job description"

    URL_list = scrape_google(query)
    
    output = {}
    
    # Html text headers h2 and h3 of interest
    article_titles = ['job description', 'job duties']
    
    # This would be the similary score threshold between article_titles and the actual page h2 and h3 titles
    threshold = 0.5
    
    for URL in URL_list:
        
        output[URL] = extract_Text(URL, article_titles, threshold)
    
    return output

In [249]:
extracting_job_description('Legislator')

{'https://www.jkentstaffing.com/job_title/legislative-assistant/': None,
 'https://www.jobbank.gc.ca/marketreport/occupation/2913/ca': ['Legislators participate in the activities of a federal, provincial, territorial or local government legislative body or executive council, band council or school board as elected or appointed members.',
  'This group performs some or all of the following duties:Enact, amend or repeal laws and regulationsParticipate in developing or amending government policies, programs or proceduresRepresent their government at local, national and international meetings and conferencesRespond to matters of concern to constituents or the general publicMay serve as minister responsible for a government department or agency and direct senior government managers and officials in the implementation of government policy and the management of that department or agency.'],
 'https://laddc.org/initiatives/community-living-and-self-determination/quality-assurance/current-initi